In [9]:

from os import environ
from redis import Redis
from time import sleep

stream_key = environ.get("STREAM", "STREAM")
producer = environ.get("PRODUCER", "project-a")


def connect_to_redis():
    hostname = environ.get("REDIS_HOSTNAME", "redis-service") 
    port = environ.get("REDIS_PORT", 6379)

    return Redis(hostname, port, retry_on_timeout=True)


def send_event(redis_connection, reference):
    count = 0

    try:
        # TODO cloud events
        # un-map the "data" wrt app package parameters
        data = {
            "subject": reference,
            "producer": producer,
            "href": reference,
        }
        resp = redis_connection.xadd(stream_key, data)
        print(resp)
        count += 1

    except ConnectionError as e:
        print(f"ERROR REDIS CONNECTION: {e}")



In [8]:

connection = connect_to_redis()
connection

<redis.client.Redis(<redis.connection.ConnectionPool(<redis.connection.Connection(host=redis-service,port=6379,db=0)>)>)>